In [40]:
from utils import (
    clean_date_of_stop,
    clean_latitude, clean_longitude,
    clean_description,
    invalid_driver_city, prefix_mapping,
    format_and_clean_geolocation, validate_geolocation, geo_to_string,
    clean_location,
    invalid_make,
    invalid_model,
    clean_time_of_stop
)
import pandas as pd
import json
import re

try:
    violations_df = pd.read_csv("./dataset/Traffic_Violations.csv")
except FileNotFoundError:
    raise FileNotFoundError("CSV file not found at ./dataset/Traffic_Violations.csv")
except Exception as e:
    raise RuntimeError(f"Unexpected error while loading CSV: {e}")

<h1><b>SeqID</b></h1>

In [41]:
try:
    # Dropping duplicate SeqID's 
    violations_df = violations_df.drop_duplicates(subset=['SeqID'], keep='first')

    violations_df = violations_df.reset_index(drop=True)

    violations_df["SeqID"] = violations_df["SeqID"].astype("string")
except Exception as e:
    raise RuntimeError(f"Error while processing SeqID column: {e}")

<h1><b>Date Of Stop</b></h1>

In [42]:
try:
    violations_df["Date Of Stop"] = violations_df["Date Of Stop"].apply(clean_date_of_stop)

    violations_df.dropna(subset=["Date Of Stop"], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Date Of Stop column: {e}")

<h1><b>Time Of Stop</b></h1>

In [43]:
try:
    violations_df["Time Of Stop"] = violations_df["Time Of Stop"].apply(clean_time_of_stop)

    violations_df.dropna(subset=["Time Of Stop"], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Time Of Stop column: {e}")

<h1><b>Agency</b></h1>

In [44]:
try:
    # Normalizing text case for consistent values
    violations_df["Agency"] = violations_df["Agency"].str.upper()
except Exception as e: 
    raise RuntimeError(f"Error while tranforming Agency: {e}")

<h1><b>SubAgency</b></h1>

In [45]:
try:
    violations_df = violations_df.rename(columns={"SubAgency": "Sub Agency"})

    # Removing extra spaces around slash
    violations_df["Sub Agency"] = violations_df["Sub Agency"].apply(lambda x: re.sub(r'\s*/\s*', '/', x))
except Exception as e: 
    raise RuntimeError(f"Error while tranforming Sub Agency: {e}")

<h1><b>Description</b></h1>

In [46]:
try:
    violations_df["Description"] = violations_df["Description"].apply(clean_description)
    
    violations_df.dropna(subset=["Description"], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Description column: {e}")

<h1><b>Location</b></h1>

In [47]:
try:
    violations_df["Location"] = violations_df["Location"].apply(clean_location)

    violations_df.dropna(subset=["Location"], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Location column: {e}")

<h1><b>Latitude & Longitude</b></h1>

In [48]:
try:
    # Converting data to float and rounding to 6 decimal places
    violations_df["Latitude"] = pd.to_numeric(violations_df["Latitude"], errors="coerce").astype("float64").round(6)
    
    violations_df["Latitude"] = violations_df["Latitude"].apply(clean_latitude)
    
    violations_df.dropna(subset=["Latitude"], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Latitude column: {e}")

try:
    # Converting data to float and rounding to 6 decimal places
    violations_df["Longitude"] = pd.to_numeric(violations_df["Longitude"], errors="coerce").astype("float64").round(6)
    
    violations_df["Longitude"] = violations_df["Longitude"].apply(clean_longitude)
    
    violations_df.dropna(subset=["Longitude"], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Longitude column: {e}")

<h1><b>Accident, Belts, Personal Injury, Property Damage, Fatal, Commercial License, HAZMAT, Commercial Vehicle, Alcohol, Work Zone, Search Conducted</b></h1>

In [49]:
# Data transforming to categorical columns which are having values True & False
cols = ["Accident", "Belts", "Personal Injury", "Property Damage", "Fatal", "Commercial License", "HAZMAT", "Commercial Vehicle", "Alcohol", "Work Zone", "Search Conducted"]

mapping = {
        'y': 'true',
        'yes': 'true',
        'n': 'false',
        'no': 'false',
        '1': 'true',
        '0': 'false',
        'FALSE': 'false',
        '': 'false'
    }

try:
    for col in cols:
        violations_df[col] = violations_df[col].replace(mapping)
        violations_df.dropna(subset=[col], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while applying mapping and dropping NaNs: {e}")


<h1><b>Search Disposition</b></h1>

In [50]:
try:
    # Fixing NaN to "Not Applicable" & Standardizing text case
    violations_df["Search Disposition"] = violations_df["Search Disposition"].apply(
        lambda x: "Not Applicable" if pd.isna(x) else str(x).title() if len(str(x)) > 3 else str(x)
    )
except Exception as e: 
    raise RuntimeError(f"Error while tranforming Search Disposition column: {e}")

<h1><b>Search Outcome</b></h1>

In [51]:
try:
    violations_df.dropna(subset=["Search Outcome"], inplace=True)

    # Standardizing text case
    violations_df["Search Outcome"] = violations_df["Search Outcome"].apply(lambda x: str(x).title())
except Exception as e: 
    raise RuntimeError(f"Error while tranforming Search Outcome column: {e}")

<h1><b>Search Reason</b></h1>

In [52]:
try:
    # Fixing NaN to "Other" & Standardizing text case
    violations_df["Search Reason"] = violations_df["Search Reason"].apply(
        lambda x: "Other" if pd.isna(x) else str(x).title() if len(str(x)) > 3 else x
    )

    violations_df["Search Reason"] = violations_df["Search Reason"].replace({"Probable Cause For Cds": "Probable Cause For CDS"})
except Exception as e: 
    raise RuntimeError(f"Error while tranforming Search Reason column: {e}")

<h1><b>Search Reason For Stop</b></h1>

In [53]:
try:
    violations_df.dropna(subset=["Search Reason For Stop"], inplace=True)

    # Removing Noise Data
    violations_df = violations_df[violations_df["Search Reason For Stop"].str.len() > 5]

    violations_df["Search Reason For Stop"] = violations_df["Search Reason For Stop"].apply(lambda x: str(x).upper())
except Exception as e:
    raise RuntimeError(f"Error while cleaning Search Reason For Stop column: {e}")

<h1><b>Search Type</b></h1>

In [54]:
try:
    # Fixing NaN to "Not Applicable" & Standardizing text case
    violations_df["Search Type"] = violations_df["Search Type"].apply(
        lambda x: "Not Applicable" if pd.isna(x) else str(x).title() if len(str(x)) > 2 else x
    )
except Exception as e:
    raise RuntimeError(f"Error while tranforming Search Type column: {e}")

<h1><b>Search Arrest Reason</b></h1>

In [55]:
try:
    # Fixing NaN to "Other" & Standardizing text case
    violations_df["Search Arrest Reason"] = violations_df["Search Arrest Reason"].apply(
        lambda x: "Other" if pd.isna(x) else str(x).title() if len(str(x)) > 3 else x
    )
except Exception as e:
    raise RuntimeError(f"Error while cleaning Search Arrest Reason column: {e}")

<h1><b>State</b></h1>

In [56]:
try:
    violations_df.dropna(subset=["State"], inplace=True)

    # Standardizing text case
    violations_df["State"] = violations_df["State"].apply(lambda x: str(x).upper())
except Exception as e:
    raise RuntimeError(f"Error while cleaning State column: {e}")

<h1><b>VehicleType</b></h1>

In [57]:
mapping = {
    "18 - Police Vehicle" : "18 - Police(Non-Emerg)",
    "28 - Electric Bicycle": "28 - Other",
    "29 - Other": "29 - Unknown"
}

try:
    violations_df = violations_df.rename(columns={"VehicleType": "Vehicle Type"})

    violations_df["Vehicle Type"] = violations_df["Vehicle Type"].replace(mapping)
    violations_df["Vehicle Type"] = violations_df["Vehicle Type"].apply(lambda x: str(x).title())

    # Splitting "Vehicle Type" into 2 column - "Vehicle Code", "Vehicle Category"
    violations_df[["Vehicle Code", "Vehicle Category"]] = violations_df["Vehicle Type"].str.split(" - ", n=1, expand=True)
    violations_df["Vehicle Code"] = violations_df["Vehicle Code"].astype(int)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Vehicle Type column: {e}")

<h1><b>Year</b></h1>

In [58]:
try:
    # Converting data to Integers
    violations_df["Year"] = pd.to_numeric(violations_df["Year"], errors="coerce").astype("Int64")

    # Marking invalid data as None
    violations_df["Year"] = violations_df["Year"].apply(lambda x: x if 1960 <= x <= 2025 else None)

    violations_df.dropna(subset=["Year"], inplace=True)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Year column: {e}")

<h1><b>Make</b></h1>

In [59]:
try:    
    # Removing all non-alphabetic characters
    violations_df["Make"] = violations_df["Make"].str.replace(r"[^A-Za-z]", "", regex=True)

    # Correcting typos, abbreviations, and spelling variations 
    with open("./mappings/make.json", "r") as f:
        mapping = json.load(f)
    violations_df["Make"] = violations_df["Make"].replace(mapping)

    # Removing invalid data
    violations_df = violations_df[~violations_df["Make"].isin(invalid_make)]

    violations_df.dropna(subset=["Make"], inplace=True)
except FileNotFoundError:
    raise FileNotFoundError("make.json not found at ./mappings/make.json")
except Exception as e:
    raise RuntimeError(f"Error while cleaning Make column: {e}")


<h1><b>Model</b></h1>

In [60]:
try:
    # Stadardizing text case and spacing
    violations_df["Model"] = violations_df["Model"].apply(
            lambda x: " ".join(str(x).split()).upper()
        )

    # Removing noise data
    violations_df = violations_df[violations_df["Model"].str.len()>1]

    # Removing invalid data
    violations_df = violations_df[~violations_df["Model"].isin(invalid_model)]

    # Correcting typos, abbreviations, and spelling variations
    with open("./mappings/model.json", "r") as f:
        mapping = json.load(f)
    violations_df["Model"] = violations_df["Model"].replace(mapping)
except FileNotFoundError:
    raise FileNotFoundError("model.json not found at ./mappings/model.json")
except Exception as e:
    raise RuntimeError(f"Error while cleaning Model column: {e}")

<h1><b>Color</b></h1>

In [61]:
mapping = {
    "BLUE, DARK" : "Dark Blue",
    "BLUE, LIGHT" : "Light Blue",
    "GREEN, LGT" : "Light Green",
    "GREEN, DK" : "Dark Green"
}

try:
    violations_df.dropna(subset=["Color"], inplace=True)

    violations_df["Color"] = violations_df["Color"].replace(mapping)
    
    # Standardizing text case
    violations_df["Color"] = violations_df["Color"].apply(lambda x: str(x).title())
except Exception as e:
    raise RuntimeError(f"Error while transforming Color column: {e}")

<h1><b>Violation Type</b></h1>

In [62]:
try: 
    # Standardizing text case
    violations_df["Violation Type"] = violations_df["Violation Type"].apply(lambda x: str(x).upper())
except Exception as e:
    raise RuntimeError(f"Error while transforming Violation Type column: {e}")

<h1><b>Charge</b></h1>

In [63]:
try:
    # Removing noise data
    violations_df = violations_df[violations_df["Charge"].str.len() > 3]

    # Standardizing text case
    violations_df["Charge"] = violations_df["Charge"].apply(lambda x: str(x).upper())
except Exception as e:
    raise RuntimeError(f"Error while cleaning Charge column: {e}")

<h1><b>Article</b></h1>

In [64]:
invalid_articles = ["Maryland Rules", "00", "BR", "1A"]
try:
    # Removing invalid articles (Unrelated articles)
    violations_df = violations_df[~violations_df["Article"].isin(invalid_articles)]
except Exception as e:
    raise RuntimeError(f"Error while cleaning Article column: {e}")

<h1><b>Contributed To Accident</b></h1>

In [65]:
try:
    # Stadardizing "Yes" & "No" accross all categorical columns
    violations_df["Contributed To Accident"] = violations_df["Contributed To Accident"].map({True: "Yes", False: "No"})
except Exception as e:
    raise RuntimeError(f"Error while converting type of 'Contributes To Accident' column: {e}")

<h1><b>Race</b></h1>

In [66]:
try:
    # Standardizing text case
    violations_df["Race"] = violations_df["Race"].apply(lambda x: str(x).upper())
except Exception as e:
    raise RuntimeError(f"Error while transforming Race column: {e}")

<h1><b>Gender</b></h1>

In [67]:
try:
    violations_df["Gender"] = violations_df["Gender"].replace({"M":"Male", "F":"Female", "U": "Unknown"})
except Exception as e:
    raise RuntimeError(f"Error while transforming Gender column: {e}")


<h1><b>Driver City</b></h1>

In [68]:
try:
    violations_df.dropna(subset=["Driver City"], inplace=True)

    # Removing noise data
    violations_df = violations_df[violations_df["Driver City"].str.len() > 3]

    # Removing invalid cities
    violations_df = violations_df[~violations_df["Driver City"].isin(invalid_driver_city)]

    # Correcting typos, abbreviations, and spelling variations
    with open("./mappings/driver_city.json", "r") as f:
        mapping = json.load(f)
    violations_df["Driver City"] = violations_df["Driver City"].replace(mapping)

    # Expanding common abbreviations
    for pattern, repl in prefix_mapping.items(): 
        violations_df["Driver City"]  = violations_df["Driver City"].str.replace(pattern, repl, regex=True)

    # Standardizing text case & spaces
    violations_df["Driver City"] = violations_df["Driver City"].apply(
            lambda x: " ".join(str(x).split()).upper()
        )
except FileNotFoundError:
    raise FileNotFoundError("driver_city.json not found at ./mappings/driver_city.json")
except Exception as e:
    raise RuntimeError(f"Error while cleaning Driver City column: {e}")

<h1><b>Driver State</b></h1>

In [69]:
try:
    # Standardizing text case
    violations_df["Driver State"] = violations_df["Driver State"].apply(lambda x: str(x).upper())
except Exception as e:
    raise RuntimeError(f"Error while transforming Driver State column: {e}")

<h1><b>DL State</b></h1>

In [70]:
try:
    violations_df.dropna(subset=["DL State"], inplace=True)

    # Standardizing text case
    violations_df["DL State"] = violations_df["DL State"].apply(lambda x: str(x).upper())
except Exception as e:
    raise RuntimeError(f"Error while cleaning DL State column: {e}")

<h1><b>Arrest Type</b></h1>

In [71]:
try:
    # Splitting "Arrest Type" into 2 columns - "Arrest Type Code", "Arrest Type Description"
    violations_df[["Arrest Type Code", "Arrest Type Description"]] = violations_df["Arrest Type"].str.split(" - ", n=1, expand=True)
except Exception as e:
    raise RuntimeError(f"Error while transforming Arrest Type column: {e}")

<h1><b>Geolocation</b></h1>

In [72]:
try:
    violations_df["Geolocation"] = violations_df["Geolocation"].apply(format_and_clean_geolocation)

    violations_df["Geolocation"] = violations_df.apply(validate_geolocation, axis=1)

    # Converting Tuple to String for database synchronization
    violations_df["Geolocation"] = violations_df["Geolocation"].apply(geo_to_string)
except Exception as e:
    raise RuntimeError(f"Error while cleaning Geolocation column: {e}")

<h1><b>Syncing violation_df with Database</b></h1>

In [73]:
try:
    # Renaming columns for database synchronization
    with open("./mappings/column_rename.json", "r") as f:
            mapping = json.load(f)
    violations_df = violations_df.rename(columns=mapping)
except FileNotFoundError:
    raise FileNotFoundError("column_rename.json not found at ./mappings/column_rename.json")
except Exception as e:
    raise RuntimeError(f"Error while renaming columns: {e}")

In [74]:
# Inserting violations data into the database
from utils.db import get_engine

try:
    engine = get_engine()

    violations_df.to_sql(
        "violations",
        engine,
        if_exists="append",
        index=False,
        method="multi",
        chunksize=5000
    )
except Exception as e:
    raise RuntimeError(f"Failed to insert violations data: {e}")
finally:
    if engine is not None: engine.dispose()